In [12]:

import os
import numpy as np 
import pandas as pd
from glob import glob
import tensorflow as tf
import tensorflow_hub as hub
from IPython.display import clear_output as cls

# Data 
from tensorflow.keras.utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator

# Data Visualization
import plotly.express as px
import matplotlib.pyplot as plt

# Model 
from keras.models import Sequential, load_model
from keras.layers import GlobalAvgPool2D as GAP, Dense, Dropout

# Callbacks 
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Pre-Trained Model
from tensorflow.keras.applications import ResNet50V2

#Directory Manipulation
import os
from sklearn.model_selection import train_test_split
import shutil

In [13]:
# Define the root directory
root_dir = 'Monkeypox Skin Image Dataset/'
# Get the class names
class_names = os.listdir(root_dir)

# Create directories for the training, validation, and testing sets
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split the filenames in each class directory into training, validation, and testing sets
for class_name in class_names:
    class_dir = os.path.join(root_dir, class_name)
    filenames = os.listdir(class_dir)
    train_filenames, test_filenames = train_test_split(filenames, test_size=0.2, random_state=42)
    train_filenames, val_filenames = train_test_split(train_filenames, test_size=0.25, random_state=42)
    
    # Create directories for the class in the training, validation, and testing sets
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)
    
    # Copy the images to the class directories in the training, validation, and testing sets
    for filename in train_filenames:
        src_path = os.path.join(class_dir, filename)
        dst_path = os.path.join(train_class_dir, filename)
        shutil.copyfile(src_path, dst_path)
    
    for filename in val_filenames:
        src_path = os.path.join(class_dir, filename)
        dst_path = os.path.join(val_class_dir, filename)
        shutil.copyfile(src_path, dst_path)
    
    for filename in test_filenames:
        src_path = os.path.join(class_dir, filename)
        dst_path = os.path.join(test_class_dir, filename)
        shutil.copyfile(src_path, dst_path)


In [14]:
root_path = 'Monkeypox Skin Image Dataset/'
class_names = sorted(os.listdir(root_path))
n_classes = len(class_names)

# Class Distribution
class_dis = [len(os.listdir(root_path + name)) for name in class_names]

# Show
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

Total Number of Classes : 4 
Class Names : ['Chickenpox', 'Measles', 'Monkeypox', 'Normal']


In [22]:
import plotly.express as px

# Plot Pie Chart
fig = px.pie(names=class_names, values=class_dis, title="Training Class Distribution", hole=0.4)

# Center the title
fig.update_layout(title={'x':0.48})

# Show the figure
fig.show()


In [16]:
# Initialize Generator
train_gen = ImageDataGenerator(rescale=1/255., rotation_range=10, horizontal_flip=True)
valid_gen = ImageDataGenerator(rescale=1/255.)
test_gen = ImageDataGenerator(rescale=1/255)

# Data Load
train_ds = train_gen.flow_from_directory(train_dir, class_mode='binary', target_size=(256,256), shuffle=True, batch_size=32) 
valid_ds = valid_gen.flow_from_directory(val_dir, class_mode='binary', target_size=(256,256), shuffle=True, batch_size=32) 
test_ds = test_gen.flow_from_directory(test_dir, class_mode='binary', target_size=(256,256), shuffle=True, batch_size=32)

Found 459 images belonging to 4 classes.
Found 155 images belonging to 4 classes.
Found 156 images belonging to 4 classes.
